<a href="https://colab.research.google.com/github/evgeny-kolonsky/gaza-news/blob/main/Death_toll.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Death toll view

get data collected from Goggle News with keyword 'Gaza'

filter 'death poll' mentions and plot on timeline plot.

In [1]:
import pandas as pd
from urllib.request import urlopen
import altair as alt

## Extract news headlines with `death toll` mention

In [133]:
url = "https://raw.githubusercontent.com/evgeny-kolonsky/gaza-news/main/gaza-news.txt"

txt = ''
data = urlopen(url).readlines()
for line in data:
  line = line.decode().lower().strip()
  try:
    date, publisher, title, origin = line.split('\t')
  except:
    continue
  title = title.replace(',','') # to read numbers such as 10,000 with comma
  if 'death toll' in title:
    numbers = [int(s) for s in title.split() if s.isdigit()]
    numbers.sort(reverse=True) # could be non-accurate if injured mentioned

    if len(numbers) > 0:
      txt += f'{date}\t{numbers[0]}\t{publisher}\t{title}\t{origin}\n'

filename = 'death-toll.tsv'
with open(filename, 'w') as f:
  f.write('date\ttoll\tpublisher\ttitle\torigin\n')
  f.write(txt)

## Load filter data as `pandas dataframe`

In [3]:
dtypes ={'date': 'str',
                  'toll': 'int',
                  'publisher': 'str',
                  'title': 'str',
                  'url': 'str'}
df  = pd.read_csv('death-toll.tsv', delimiter='\t',
                  dtype=dtypes, date_format="%Y-%m-%d")
df['date'] = pd.to_datetime(df['date'])
df.head()

,date,toll,publisher,title,origin
0,2023-10-07,1200,the associated press,israel battles hamas militants as death toll n...,https://apnews.com/article/israel-palestinians...
1,2023-10-17,3478,anadolu agency | english,palestinian death toll in gaza conflict rises ...,https://news.google.com/rss/articles/cbmifwh0d...
2,2023-10-18,4000,reliefweb,israel destroys a quarter of northern gaza str...,https://reliefweb.int/report/occupied-palestin...
3,2023-10-22,5000,united nations sustainable development group,unsdg | gaza death toll passes 5000 with no ce...,https://unsdg.un.org/latest/stories/gaza-death...
4,2023-10-22,5000,un news,israel-palestine: gaza death toll passes 5000 ...,https://news.google.com/rss/articles/cbmilgh0d...


## Plot timeline data

In [4]:
alt.Chart(df).mark_point().encode(
    x='date',
    y='toll',
    tooltip=['date', 'publisher', 'title', 'origin'],
).interactive()

alt.Chart(...)